In [1]:
import os
import openai
from futureturngpt.model import TurnGPT, TurnGPTWandbCallbacks
from os.path import join
import re
import pdb
from futureturngpt.plot_utils import plot_trp
import wandb
import matplotlib.pyplot as plt
import torch

In [2]:
model_type = 'futureturngpt'

future_turngpt = True
if future_turngpt:
    chpt_path = '../runs/futureTurnGPT/futureTurnGPT_3qrpr59m_epoch=3_val_loss=0.2666.ckpt'
    chpt = join(
            chpt_path
        )
    model = TurnGPT.load_from_checkpoint(chpt)
else:
    from original_turngpt.model import originalTurnGPT, originalTurnGPTWandbCallbacks
    chpt = join(
        "/home/binger/repos/TurnGPT/runs/TurnGPT/TurnGPT_2cr6pudn/epoch=10_val_loss=1.7908.ckpt"
    )
    model =  originalTurnGPT.load_from_checkpoint(chpt)
turn_list = [
        #['What did you do yesterday?', 'I went hiking with my friends John and Mary. What did you do yesterday?', 'I was painting the wall of my garage.'],
        #['What did you do yesterday?', 'I went hiking with my friends John and Mary.', 'That sounds cool!'],
        #['What did you do yesterday?', 'I went hiking with my friends John and Mary. What did you do yesterday?', 'That sounds cool!'],
        #['What did you do yesterday?', 'I went hiking with my friends John and Mary. What did you do yesterday?', 'Are you okay?'],
        #['What did you order?', 'What we always have for brunch here, tuna sandwiches, fries, pudding, and coffee.', 'That sounds good!'],
        #['What did you order?', 'What we always have for brunch here, tuna sandwiches, fries, pudding, and coffee.', 'Are you okay?'],
        #['What did you order?', 'What we always have for brunch here, do you want anything else today?', 'No, that sounds good!']
        #['Did you and Mary meet yesterday?','Yes, we met in the park.',"That's great, I'm glad to hear that! when will you meet again?",'tomorrow.']
        ['Yesterday we met in the park.','Okay, when will you meet again?','tomorrow.']
        #["peter enough with your computer games go do your homework now","can't i play more?","no, stop playing computer games", "mom i'll be finished soon","peter, if you don't turn off your computer then i won't allow you to play it again starting next week."]
        #["hi brittany, what are you doing with all of your clothes on your bed?","i'm trying to decide what to wear to school the first day","oh mom didn't tell you?"," didn't tell me what?"," this school you're going to is going to make your life easy"," what are you talking about brother spill it"," uniforms sis, no more worrying about appearances","you mean i have to wear the same thing every day mom"]
        #["hello sir i'm ready for you","is it my turn?","yes, please sit on the chair. how do you want to have your hair cut?","not too long, cut a little off behind and on both sides too","ok, now lean back a little and keep still. i'm going to shave your face."]
    ]

model.transformer.config.output_attentions = True

######################################################################
LOAD CHECKPOINT TOKENIZER
Loaded tokenizer
PreTrainedTokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_len=1e+30, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<ts>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<speaker1>', '<speaker2>', '<future>']})
Resized weights
######################################################################
1


In [3]:
turn_list = [
    ["Hi, I'd like to book an appointment with Dr. X.",
     "I'm sorry, Dr. X isn't available this month. Would you like to book an appointment with Dr. Y or Dr. Z instead?",
     "Are there any other doctors available this month?"
    ],
    ["Hi, I'd like to book an appointment with Dr. X.",
     "I'm sorry, Dr. X isn't available this month. Would you like to book an appointment with Dr. Y or Dr. Z instead?",
     "I'd like to book an appointment with Dr. Y then, thank you."
    ],
]

In [4]:
out = model.string_list_to_trp_inputid(turn_list)

/home/binger/repos/futureTurnGPT/futureturngpt/model.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp_inp.append(torch.tensor(inp))
/home/binger/repos/futureTurnGPT/futureturngpt/model.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp_sp.append(torch.tensor(sp))


In [5]:
out.keys()

odict_keys(['loss', 'mc_loss', 'logits', 'mc_logits', 'past_key_values', 'attentions', 'probs', 'trp_probs', 'tokens', 'input_ids', 'trp_proj'])

In [6]:
attns = out['attentions'][-1]

In [7]:
input_ids = out['input_ids']
input_ids.shape

torch.Size([2, 51])

In [8]:
input_ids[0]

tensor([  389,   612,   597,   584,  7519,  1695,   428,  1227, 50257,  5303,
         1312,  1549,   588,   284,  1492,   281, 12557,   351,  1553,  2124,
        50257,  1312,  1101,  7926,  1553,  2124,  2125,   470,  1695,   428,
         1227,   561,   345,   588,   284,  1492,   281, 12557,   351,  1553,
          331,   393,  1553,  1976,  2427, 50257, 50256, 50256, 50256, 50256,
        50256])

In [9]:
input_ids[1]

tensor([ 1312,  1549,   588,   284,  1492,   281, 12557,   351,  1553,   331,
          788,  5875,   345, 50257,  5303,  1312,  1549,   588,   284,  1492,
          281, 12557,   351,  1553,  2124, 50257,  1312,  1101,  7926,  1553,
         2124,  2125,   470,  1695,   428,  1227,   561,   345,   588,   284,
         1492,   281, 12557,   351,  1553,   331,   393,  1553,  1976,  2427,
        50257])

In [8]:
ts_idx = [(sublist == model.tokenizer.eos_token_id).nonzero(as_tuple=True)[0] for sublist in input_ids]

In [11]:
ts_idx

[tensor([ 8, 20, 45]), tensor([13, 25, 50])]

In [ ]:
ts = ts_idx[0][0]

In [ ]:
attn=attns[0,:,ts,:]

In [ ]:
attn.shape

In [ ]:
attns[0,0,0]

In [ ]:
sum(attns[0,0,-1,:ts])

In [ ]:
sum(attns[0,0,-1])

In [9]:
@torch.no_grad()
def percent_future_attention (attns,input_ids):
    '''
    attns = [batch_size, n_head, len_dialog, len_dialog]
    '''
    future_attentions = [] # one value for each dialog (average future attention by head)
    future_attentions_all_heads = [] # n_head value for each dialog
    ts_idx = [(sublist == model.tokenizer.eos_token_id).nonzero(as_tuple=True)[0] for sublist in input_ids]
    n_heads = attns.shape[1]
    for i, attn in enumerate(attns):
        future_ts = ts_idx[i][0]
        attn_future_all = [sum(attns[i,j,-1,:future_ts]) for j in range(12)]
        future_attentions.append(sum(attn_future_all)/n_heads)
        future_attentions_all_heads.append(attn_future_all)
    
    return torch.FloatTensor(future_attentions), torch.FloatTensor(future_attentions_all_heads)

In [10]:
fut_attns, fut_attns_heads = percent_future_attention(attns, out['input_ids'])

In [16]:
torch.sum(attns[0,:,:,:8],0)/12

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7434, 0.2566, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6699, 0.1868, 0.1434, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6337, 0.1404, 0.1352, 0.0908, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7246, 0.0705, 0.0801, 0.0780, 0.0468, 0.0000, 0.0000, 0.0000],
        [0.7266, 0.0350, 0.0313, 0.0304, 0.1016, 0.0752, 0.0000, 0.0000],
        [0.6820, 0.0204, 0.0325, 0.0358, 0.1090, 0.0737, 0.0466, 0.0000],
        [0.6499, 0.0313, 0.0284, 0.0258, 0.1015, 0.0946, 0.0287, 0.0398],
        [0.5167, 0.0594, 0.0414, 0.0438, 0.0827, 0.0597, 0.0190, 0.0590],
        [0.6136, 0.0369, 0.0267, 0.0251, 0.0404, 0.0279, 0.0151, 0.0378],
        [0.5373, 0.0298, 0.0266, 0.0271, 0.0610, 0.0552, 0.0118, 0.0807],
        [0.5476, 0.0364, 0.0345, 0.0309, 0.0598, 0.0293, 0.0122, 0.0455],
        [0.5046, 0.0386, 0.0354, 0.0318, 0.0838, 0.0594, 0.0163, 0.0529],
        [0.5057, 0.0276, 0.0216, 0.018

In [58]:
average_attn = torch.sum(attns[0,:,:,:8],0)/attns.shape[1]

In [33]:
input_ids[0][:9]

tensor([  389,   612,   597,   584,  7519,  1695,   428,  1227, 50257])

In [29]:
fut_attn_by_tok = sum(average_attn, 0)/average_attn.shape[0]

In [60]:
fut_attn_by_tok

tensor([0.5289, 0.0310, 0.0208, 0.0159, 0.0297, 0.0218, 0.0087, 0.0188])

the fist token always takes too much attention. so it always stops at the first token.

In [13]:
@torch.no_grad()
def calculate_main_attention (attns, future_attentions,ts_idx, percent = 0.9):
    '''
    attention: one per each converstation, only last one permutated
    [batch_size, n_head, len_dialog, len_dialog]
    '''
    furthest_indices_all = []
    rel_furthest_indices_all = []
    future_attentions_thresholds = future_attentions * percent
    for j, attn in enumerate(attns):
        future_ts = ts_idx[j][0]
        current_ts = ts_idx[j][-1]
        future_attn_threshold = future_attentions_thresholds[j]
        # average over attention heads
        # attn = [n_head, len_dialog, len_fut_utt]
        average_attn = torch.sum(attn[:,:,:future_ts],0)/attn.shape[0]
        # average over y axis
        # fut_attn_by_tok = one val for each tok in future utt
        #fut_attn_by_tok = sum(average_attn, 0)/average_attn.shape[0]
        fut_attn_by_tok = average_attn[current_ts]
        if sum(fut_attn_by_tok) < future_attn_threshold:
            # doesn't reach threshold at all
            furthest_indices_all.append(-1)
            rel_furthest_indices_all.append(-1)
        else: # reaches threshold at some point
            cumulative_attn = 0
            i = 0
            while i < len(fut_attn_by_tok) and cumulative_attn < future_attn_threshold:
                cumulative_attn += fut_attn_by_tok[i]
                if cumulative_attn > future_attn_threshold:
                    print(cumulative_attn)
                    print(future_attentions_thresholds)
                    print('-----')
                    furthest_indices_all.append(i)
                    rel_furthest_indices_all.append(i/len(fut_attn_by_tok))
                    break
                else:
                    i+=1
    return torch.FloatTensor(furthest_indices_all), torch.FloatTensor(rel_furthest_indices_all)

In [14]:
furthestidx, relidx = calculate_main_attention(attns, fut_attns, ts_idx)

tensor(0.3348)
tensor([0.4867, 0.3196])
-----


In [15]:
furthestidx

tensor([-1., 11.])

In [16]:
relidx

tensor([-1.0000,  0.8462])

In [42]:
torch.FloatTensor(fut_attns)*0.9

tensor([0.4867, 0.3196])

In [ ]:
attns[0,:,:,:ts_idx]

In [41]:
attns[0,:,:,:8].shape

torch.Size([12, 51, 8])

In [ ]:
e

In [ ]:
sum(attns[2,0,-1,:13])

In [ ]:
@torch.no_grad()
def string_list_to_trp_inputid(self, string_or_list, add_post_eos_token=False, use_label=True,**model_kwargs):
    t = self.tokenize_strings(string_or_list, add_post_eos_token=add_post_eos_token)
    if use_label:
        lm_labels = self.get_labels(t["input_ids"], mask=t['attention_mask'])
    proj_labels = None
    if self.trp_projection_steps > 0:
        proj_labels = self.get_projection_labels(
            t["input_ids"], mask=t["attention_mask"]
        )
    # Model
    if not use_label:
        out = self(t["input_ids"], speaker_ids=t["speaker_ids"], **model_kwargs)
    else:
        out = self(t["input_ids"], 
                   speaker_ids=t["speaker_ids"], 
                   labels=lm_labels,
                   mc_labels=proj_labels,
                   **model_kwargs)        
    out["probs"] = out["logits"].softmax(dim=-1)
    out["trp_probs"] = self.get_trp(out["probs"])
    out["tokens"] = self.get_tokens(t["input_ids"])
    out['input_ids'] = t['input_ids']
    if "mc_logits" in out:
        out["trp_proj"] = out["mc_logits"].sigmoid()
    #pdb.set_trace()
    return out  

In [20]:
heads = [sum(attns[0,i,-1,:8]) for i in range(12)]

In [22]:
sum(heads)/len(heads)

tensor(0.5407)

In [ ]:
sum(heads)/12